In [1]:
import { load } from "dotenv"
const env = await load({
  envPath: ".env.local",
})

const process = { env }

const chatOptions = {
  openAIApiKey: process.env.Tongyi_API_KEY,
  temperature: 0,
  modelName: "qwen-plus",
  configuration: {
    baseURL: process.env.BASE_URL,
  },
}
const tongyiChatOptions = {
  alibabaApiKey: process.env.Tongyi_API_KEY,
  temperature: 1.5,
  modelName: "qwen-plus",
};


In [2]:
import {
  ChatPromptTemplate,
  PromptTemplate,
  MessagesPlaceholder,
} from "@langchain/core/prompts"
import { ChatAlibabaTongyi } from "@langchain/community/chat_models/alibaba_tongyi"
import {
  RunnableSequence,
  RunnableWithMessageHistory,
} from "@langchain/core/runnables"
import { StringOutputParser } from "@langchain/core/output_parsers"
import { DynamicStructuredTool, DynamicTool } from "@langchain/core/tools"
import { AgentExecutor, createOpenAIToolsAgent } from "langchain/agents"
import { ChatMessageHistory } from "langchain/stores/message/in_memory"
import z from "zod"


[WARN]: You have enabled LangSmith tracing without backgrounding callbacks.
[WARN]: If you are not using a serverless environment where you must wait for tracing calls to finish,
[WARN]: we suggest setting "process.env.LANGCHAIN_CALLBACKS_BACKGROUND=true" to avoid additional latency.
[WARN]: You have enabled LangSmith tracing without backgrounding callbacks.
[WARN]: If you are not using a serverless environment where you must wait for tracing calls to finish,
[WARN]: we suggest setting "process.env.LANGCHAIN_CALLBACKS_BACKGROUND=true" to avoid additional latency.


In [3]:
const mbtiInfo = JSON.parse(await Deno.readTextFile("./mbti-info.json"));
console.log("🚀 ~ mbtiInfo:", mbtiInfo);

const mbtiList = Object.keys(mbtiInfo);

🚀 ~ mbtiInfo: {
  istj: "严肃、安静、藉由集中心志与全力投入、及可被信赖获致成功。行事务实、有序、实际、逻辑、真实及可信赖十分留意且乐于任何事（工作、居家、生活均有良好组织及有序。负责任。照设定成效来作出决策且不畏阻挠与闲言会坚定为之。重视传统与忠诚。传统性的思考者或经理。",
  isfj: "安静、和善、负责任且有良心。行事尽责投入。安定性高，常居项目工作或团体之安定力量。愿投入、吃苦及力求精确。兴趣通常不在于科技方面。对细节事务有耐心。忠诚、考虑周到、知性且会关切他人感受。致力于创构有序及和谐的工作与家庭环境。",
  infj: "因为坚忍、创意及必须达成的意图而能成功。会在工作中投注最大的努力。默默强力的、诚挚的及用心的关切他人。因坚守原则而受敬重。提出造福大众利益的明确远景而为人所尊敬与追随。追求创见、关系及物质财物的意义及关联。想了解什么能激励别人及对他人具洞察力。光明正大且坚信其价值观。有组织且果断地履行其愿景。",
  intj: "具强大动力与本意来达成目的与创意—固执顽固者。有宏大的愿景且能快速在众多外界事件中找出有意义的模范。对所承负职务，具良好能力于策划工作并完成。具怀疑心、挑剔性、独立性、果决，对专业水准及绩效要求高。",
  istp: "冷静旁观者—安静、预留余地、弹性及会以无偏见的好奇心与未预期原始的幽默观察与分析。有兴趣于探索原因及效果，技术事件是为何及如何运作且使用逻辑的原理组构事实、重视效能。擅长于掌握问题核心及找出解决方式。分析成事的缘由且能实时由大量资料中找出实际问题的核心。",
  isfp: "羞怯的、安宁和善地、敏感的、亲切的、且行事谦虚。喜于避开争论，不对他人强加已见或价值观。无意于领导却常是忠诚的追随者。办事不急躁，安于现状无意于以过度的急切或努力破坏现况，且非成果导向。喜欢有自有的空间及照自订的时程办事。",
  infp: "安静观察者，具理想性与对其价值观及重要之人具忠诚心。希外在生活形态与内在价值观相吻合。具好奇心且很快能看出机会所在。常担负开发创意的触媒者。除非价值观受侵犯，行事会具弹性、适应力高且承受力强。具想了解及发展他人潜能的企图。想作太多且作事全神贯注。对所处境遇及拥有不太在意。具适应力、有弹性除非价值观受到威胁。",
  intp: "安静、自持、弹性及具适应力。特别喜

In [4]:
const prompt = ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个共情能力非常强的心理医生，并且很了解MBTI（迈尔斯-布里格斯性格类型指标)的各种人格类型，你的任务是根据来访者的 MBTI 和问题，给出针对性的情感支持，你的回答要富有感情、有深度和充足的情感支持，引导来访者乐观积极面对问题",
  ],
  [
    "human",
    "用户的 MBTI 类型是{type}, 这个类型的特点是{info}, 他的问题是{question}",
  ],
])

const model = new ChatAlibabaTongyi(tongyiChatOptions)
const mbtiChain = RunnableSequence.from([
  prompt,
  model,
  new StringOutputParser(),
])


In [5]:
const mbtiTool = new DynamicStructuredTool({
  name: "get-mbti-chat",
  schema: z.object({
    type: z.enum(mbtiList).describe("用户的 MBTI 类型"),
    question: z.string().describe("用户的问题"),
  }),
  func: async ({ type, question }) => {
    const info = mbtiInfo[type]

    const res = await mbtiChain.invoke({ type, question, info })
    return res
  },
  description: "根据用户的问题和 MBTI 类型，回答用户的问题",
})


In [6]:
const tools = [mbtiTool]

const agentPrompt = await ChatPromptTemplate.fromMessages([
  [
    "system",
    "你是一个用户接待的 agent，通过自然语言询问用户的 MBTI 类型和问题，直到你有足够的信息调用 get-mbti-chat 来回答用户的问题",
  ],
  new MessagesPlaceholder("history_message"),
  ["human", "{input}"],
  new MessagesPlaceholder("agent_scratchpad"),
])

const llm = new ChatAlibabaTongyi(tongyiChatOptions)
const agent = await createOpenAIToolsAgent({
  llm,
  tools,
  prompt: agentPrompt,
})
const agentExecutor = new AgentExecutor({
  agent,
  tools,
})


In [7]:
const messgaeHistory = new ChatMessageHistory()

const agentWithChatHistory = new RunnableWithMessageHistory({
  runnable: agentExecutor,
  getMessageHistory: () => messgaeHistory,
  inputMessagesKey: "input",
  historyMessageKey: "history_message",
})
